### WEB Scraping - OPERADORES ECOLÓGICOS ANDALUCÍA  

@SARA BELÉN RAMOS GONZÁLEZ y DANIEL JESÚS CRUZ GARZÓN


### <i> Requisitos para que funcione el script </i>

+ Selenium (con el driver gecko) 

    `pip install selenium`
    
    https://github.com/mozilla/geckodriver/releases
    
    El ejectuable geckodriver.exe debe estar alojado en el directorio de trabajo del notebook o agregarlo a la variable PATH
    
    
+ Katalon Recorder (plugin para firefox que emula la navegacion humana y graba en python esta navegación)

    https://www.katalon.com/resources-center/blog/katalon-automation-recorder/

+ Habrá que definir el path correspondiente en cada caso

### Descripción

Se recorre por cada elemento del listado de provincias de la Comunidad Autónma de Andalucía todas las páginas con la información que queremos extraer. Para ello, implementaremos un bucle for que recorre provincias con un bucle while para recorrer paginas hasta que se acaben. 
La URL de la pagina elegida para hacer scraping es: https://servicio.mapama.gob.es/regoe/Publica/Operadores.aspx

### Inconvenientes

#### * Primer inconveniente

El enlace con "..." para avanzar a los siguientes enlaces tiene la forma de:

```python 
driver.find_element_by_link_text('...').click() # Hacemos click en la pagina de continuacion
```
sin embargo cuando se tienen dos enlaces con "..." (uno para avanzar y otro para volver atrás), produce que si utilizamos la misma forma que la anterior vuelva a los enlaces ya pasados y este bucle sea infinito. Por lo tanto, el enlace con "..." de avance cambia y es el siguiente:

```python
driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='...'])[1]/following::a[13]").click()
```

#### * Segundo inconveniente

Para la provincia de Almería surge error al no encontrar la tabla de información debido a que tarda en actualizarse la página después de haber producido una acción de click (siguiente provincia o página). Hay dos formas de solucionarlo:

- Si la pagina siguiente tiene un tag con id distinto al de la página de la cual proviene podemos esperar que cargue hasta que encuentre este id en la página que está cargando:

```python
# Tenemos el id de la siguiente página de carga
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "CphCuerpoPagina_GVCertificados_btnCertificadoES_0"))
```

- Si la página es prácticamente igual que la anterior y sólo cambia el contenido de la tabla, podemos hacer un sleep forzado a un tiempo constante.

```python
time.sleep(20)
```

## SCRIPT

### Importamos los módulos necesarios

In [ ]:
# Importamos los módulos necesarios
import os
from _csv import reader

# Aprovechamos y nos situamos en el directorio donde extraeremos nuestros datos
os.chdir("C:\\Users\\sara.ramos\\Anaconda3\\VS_NOTEBOOKS\\")  # colocamos nuestro path correspondiente en cada caso

# Importamos el resto de módulos necesarios
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import logging

### Funciones auxiliares

In [ ]:

# Definimos la función de espera de página
def espera_nueva_pagina():
    global driver
    try:  # Comprobamos que carga bien la pagina, condicionando que aparezca una seccion de la tabla con id
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "CphCuerpoPagina_GVCertificados_btnCertificadoES_0"))
        )
    except:
        print("Fallo al cargar la página")

# Definimos la función de hacer click en la página siguiente
def click_sig_pag():
    global driver
    global ultima_pagina
    global siguiente_pagina
    global logging

    try:  # Condicionamos a que la página cargue
        driver.find_element(By.LINK_TEXT, str(siguiente_pagina)).click()
    except:
        print("Esta página no existe: " + str(siguiente_pagina) + " probando siguiente pagina + 1")
        # Almacenamos el paso llevado a cabo en un archivo de texto a modo de log
        logging.info("Esta página no existe: " + str(siguiente_pagina) + " probando siguiente pagina + 1")
        if siguiente_pagina > int(ultima_pagina):
            driver.find_element(By.LINK_TEXT, str(siguiente_pagina + 1)).click()
            siguiente_pagina += 1

# Definimos la función para exportar los datos en la tabla correspondiente
def exportInfoTable(i, soup, outputFile):
    n_filas = 0
    cuenta = 0

    for sub_tag_i in soup.find_all('tr'):
        cuenta += 1
        # print ("PRUEBA DE SUB_TAG_I",sub_tag_i,"tag:", cuenta) #PRUEBAS OUTPUT
        # print ("LEN SUB_TAG_I LONGITUD TOTAL",len(sub_tag_i),"tag:", cuenta) #PRUEBAS OUTPUT

        global logging

        row = ""
        row_1 = ""
        row_2 = ""

        if len(sub_tag_i) >= 1:
            if len(row_1) > 0:  # Solo volcamos si la fila no esta vacia
                n_filas += 1
                outputFile.write(row_1)
                outputFile.flush()

            for subtag_x in sub_tag_i.find_all('th'):
                row_x = re.search('>(.+?)</th>', str(subtag_x)).group(1)
                row_2 += '"' + row_x + '";'

            if i == 0:  # re.sub(r"\s+", "", str(row_2)) != re.sub(r"\s+", "", nombres) and len(re.sub(r"\s+", "", str(row_2))) > 0:
                outputFile.write(row_2.replace('" ";', ''))
                outputFile.flush()

            for sub_tag_j in sub_tag_i.find_all('td'):
                sub_tag_j = sub_tag_j.text.replace(";", "").replace('"', '').replace("\n",
                                                                                     " ").strip()  # si contiene un caracter separador de columna o comilla lo quito
                row += '"' + sub_tag_j + '";'

            row = row[0:len(row) - 1]  # Quitamos el último separador ';"' innecesario
            row += "\n"

            if (len(row) > 0 and i == 0) or (len(row) > 1 and i != 0):  # Solo volcamos si la fila no esta vacia
                n_filas += 1
                if not re.match(r'^["]\d+.*|^["][.]{3}.*|^["]Visualizar.*',
                                row):  # No incorporamos las filas con los numeros de página
                    outputFile.write(row.replace('"";', ''))
                    outputFile.flush()  # Limpiamos el buffer interno

    print("Numero de filas extraidas: ", str(n_filas))  # OUTPUT FILAS


# Definimos la variable con el conjunjto de los hombres de las provincias disponibles            
prov_name = ['Almería', 'Cádiz', 'Córdoba', 'Granada', 'Huelva', 'Jaén', 'Málaga', 'Sevilla']

# Creamos el archivo donde vamos insertando los nuevos registros
outputFile = open("./outputFile_Andalucia_completo.csv", "a")
es_primer_grupo_paginas = True
siguiente_pagina = 1
soup_text_ant = ""

# También crearemos un archivo para almacenar el log de nuestra ejecución
logging.basicConfig(filename='log_Andalucia_completo.log', level=logging.INFO)



### MAIN SCRIPT

In [ ]:

# -------------------------#
#      >>>>> MAIN <<<<<    #
# -------------------------#

# Creamos una instancia al navegador firefox
driver = webdriver.Firefox()
driver.implicitly_wait(60)

# Entramos en la pagina
driver.get("https://servicio.mapama.gob.es/regoe/Publica/Operadores.aspx")

# Nos situamos en el desplegable de CCAA
driver.find_element(By.ID,"CphCuerpoPagina_ddlCCAA").click()

# Seleccionamos la CCAA "Andalucía"
Select(driver.find_element(By.ID,"CphCuerpoPagina_ddlCCAA")).select_by_visible_text('Andalucía')

# Recorremos todas las provincias del listado definiendo el rango total
for i in range(1, (len(prov_name)+1)):
    es_primer_grupo_paginas = False
    siguiente_pagina = 1

    option = i + 1
    provincia = prov_name[i - 1]
    print("Provincia: " + provincia + " option: " + str(option-1) + "\n\n")
    
    # if  provincia == 'Almería' : # PRUEBAS SIN ALMERÍA PARA AGILIZAR
    #     continue    # continue here

    # Seleccionamos una provincia
    Select(driver.find_element(By.ID,"CphCuerpoPagina_ddlProvincias")).select_by_visible_text(provincia)

    # Hacemos click en la seleccion de la provincia
    driver.find_element(By.XPATH,
        "(.//*[normalize-space(text()) and normalize-space(.)='Provincia'])[1]/following::option[" + str(
            option) + "]").click()

    # Hacemos click en el boton de buscar
    driver.find_element(By.ID, "CphCuerpoPagina_btnBuscar").click()
    # Almacenamos el paso llevado a cabo en un archivo de texto a modo de log
    logging.info('Click en buscar ' + provincia)
    espera_nueva_pagina()

    index_pg = 0  # Índice del nÚmero de páginas cargada

    while True:
        try:
            # Condicionamos la provincia que da error por espera de página
            if (provincia == 'Almería'):
                print("Esperando actualizacion de la pagina por 20 segundos...")
                time.sleep(20)

            # Extraemos el html del resultado de la acción anterior
            soup = BeautifulSoup(driver.page_source, 'lxml')

            # Extraemos el contenido de la tabla de la pagina y exportamos a CSV
            exportInfoTable(index_pg, soup, outputFile)
            # Almacenamos el paso llevado a cabo en un archivo de texto a modo de log
            logging.info('Extraccion tabla de ' + provincia + ' pagina ' + str(siguiente_pagina))

            # Extraemos la información de los links a las siguientes páginas
            soup_pag = soup.findAll('tr', {"class": "GridViewPaginacion"})  # Nos situamos en la tabla de paginaciones
            soup_pag = soup_pag[0].findAll('a')  # Extraemos todos los enlaces a cada número de página
            ultima_pagina = soup_pag[len(soup_pag) - 2].text
            continuacion_pagina = soup_pag[len(soup_pag) - 1].text
            print("Siguiente pág: " + str(siguiente_pagina) + " // Última pag: " + str(
                ultima_pagina) + " // Continuación pág: " + continuacion_pagina)
            print(50 * "-")

            # Si no hay enlace de continuación entonces la ultima página es numérica
            if continuacion_pagina != '...':
                ultima_pagina = continuacion_pagina

            siguiente_pagina += 1

            # Si la siguiente pagina sobrepasa la última página numerada entonces definimos su acción
            if siguiente_pagina > int(ultima_pagina):

                # si la última página es la de continuacion ""...""
                if continuacion_pagina == '...':
                    if es_primer_grupo_paginas == True:
                        driver.find_element(By.LINK_TEXT,'...').click()  # Hacemos click en la página de continuación
                        es_primer_grupo_paginas = False
                    else:  # cuando hay dos enlaces con "..." el de la derecha es el del siguiente xpath:
                        try:
                            driver.find_element(By.XPATH,
                                "(.//*[normalize-space(text()) and normalize-space(.)='...'])[1]/following::a[13]").click()
                        except Exception as inst:
                            driver.find_element(By.LINK_TEXT,
                                '...').click()  # Hacemos click en la página de continuación

                    # siguiente_pagina += 1 # sumamos uno mas paa saltar 2 puesto que la "..." si cuenta
                else:
                    break
            else:

                # Hacemos click en la siguiente página numérica
                # click_sig_pag()
                print("hago click en: " + str(siguiente_pagina))
                driver.find_element(By.LINK_TEXT, str(siguiente_pagina)).click()
            index_pg += 1

        except:
            print('Error en ' + provincia + ' pagina ' + str(siguiente_pagina))
            # Almacenamos el paso llevado a cabo en un archivo de texto a modo de log
            logging.info('Error en ' + provincia + ' pagina ' + str(siguiente_pagina))

outputFile.close()
driver.quit()


